<a href="https://colab.research.google.com/github/Captaingreenbeard/DataAnalyticsLab_SEM7/blob/main/lab6_email_classification_encoding_and_vectorization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from tensorflow.keras import losses, models, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.callbacks import (EarlyStopping, LearningRateScheduler,
                             ModelCheckpoint, TensorBoard, ReduceLROnPlateau)
from tensorflow.keras.layers import (Convolution2D, Dense, Dropout, GlobalAveragePooling2D, 
                              GlobalMaxPool2D, Input, MaxPool2D, concatenate, Activation,  
                              MaxPooling2D,Flatten,BatchNormalization, Conv2D,AveragePooling2D)
from tensorflow.keras.utils import Sequence, to_categorical
from sklearn.datasets import load_iris 
from sklearn.datasets import make_moons 
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import utils
from matplotlib import pyplot as plt

import numpy as np
import pandas as pd
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

In [17]:
data = pd.read_csv("emails.csv")

is_null = pd.isnull(data).sum()
print('The number of empty values by column:')
print(is_null)

The number of empty values by column:
text    0
spam    0
dtype: int64


In [18]:
data.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


In [19]:
data.columns

Index(['text', 'spam'], dtype='object')

#lets try only with the encoder

In [20]:

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(data[["text"]])
data[["text"]] = enc.transform(data[["text"]])

In [21]:
Selected_features = ['text']

X = data[Selected_features]
y = data['spam']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

#testing the correlation
#X_train.shape
#data.spam.describe()
#(data.spam == 0).sum()
#sns.pairplot(data[Selected_features].corr())
#plt.show()

In [9]:
#lets expriment with the NLP preprocessing stuff

In [22]:
data.head()

,text,spam
0,2296.0,1
1,5184.0,1
2,5276.0,1
3,75.0,1
4,854.0,1


In [23]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='ovr', max_iter=10000)
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
log_regr_score1 = clf.score(X_test, y_test)
print('log_regr_score: ',log_regr_score1)
confusion_matrix(y_test, moonsY_pred)

log_regr_score:  0.74231843575419


array([[1063,    0],
       [ 369,    0]])

In [24]:
clf = DecisionTreeClassifier(max_depth=10)
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
dec_tree_score1 = clf.score(X_test, y_test)
print('dec_tree_score: ',dec_tree_score1)
confusion_matrix(y_test, moonsY_pred)

dec_tree_score:  0.8121508379888268


array([[1004,   59],
       [ 210,  159]])

In [25]:
clf = RandomForestClassifier(n_estimators=100, max_depth=2,random_state=0)
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
rand_frst_score1 = clf.score(X_test, y_test)
print('random_forest_score: ',rand_frst_score1)
confusion_matrix(y_test, moonsY_pred)

random_forest_score:  0.7506983240223464


array([[1063,    0],
       [ 357,   12]])

In [26]:
clf = KNeighborsClassifier(n_neighbors=3)
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
KNbrs_score1 = clf.score(X_test, y_test)
print('KNbrs_score: ',KNbrs_score1)
confusion_matrix(y_test, moonsY_pred)

KNbrs_score:  0.8442737430167597


array([[982,  81],
       [142, 227]])

In [27]:
clf = SVC(gamma='auto')
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
SVM_score1 = clf.score(X_test, y_test)
print('SVM_score: ',SVM_score1)
confusion_matrix(y_test, moonsY_pred)

SVM_score:  0.7946927374301676


array([[1044,   19],
       [ 275,   94]])

In [28]:
clf = GaussianNB()
clf = clf.fit(X_train, y_train)
moonsY_pred = clf.predict(X_test)
NB_score1 = clf.score(X_test, y_test)
print('NB_score ',NB_score1)
confusion_matrix(y_test, moonsY_pred)

NB_score  0.74231843575419


array([[1063,    0],
       [ 369,    0]])

#now lets try with vectorization

In [56]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt


In [57]:
data = pd.read_csv("emails.csv")

is_null = pd.isnull(data).sum()
print('The number of empty values by column:')
print(is_null)

The number of empty values by column:
text    0
spam    0
dtype: int64


In [58]:
X = data['text']

Y = data['spam']

In [59]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

In [60]:
feature_extraction = TfidfVectorizer(min_df = 1, stop_words='english', lowercase='True')

X_train_transformed = feature_extraction.fit_transform(X_train)
X_test_transformed = feature_extraction.transform(X_test)

In [63]:
model = LogisticRegression()

In [64]:
# training the Logistic Regression model with the training data
model.fit(X_train_transformed, Y_train)

LogisticRegression()

In [65]:
# prediction on training data

prediction_on_training_data = model.predict(X_train_transformed)
accuracy_on_training_data = accuracy_score(Y_train, prediction_on_training_data)

In [66]:
print('Accuracy on training data : ', accuracy_on_training_data *100)

Accuracy on training data :  99.65080750763859


In [67]:
# prediction on test data

prediction_on_test_data = model.predict(X_test_transformed)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [68]:
print('Accuracy on test data : ', accuracy_on_test_data*100)

Accuracy on test data :  98.34205933682374
